In [1]:
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")

In [4]:
from qdrant_client.models import Distance, VectorParams, PointStruct

In [3]:
client.recreate_collection(
    collection_name="clip_image_product",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
)

True

In [5]:
import numpy as np

In [6]:
vectors = np.random.rand(100, 512)
client.upsert(
    collection_name="clip_image_product",
    points=[
        PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
        )
        for idx, vector in enumerate(vectors)
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
query_vector = np.random.rand(512)
hits = client.search(
    collection_name="clip_image_product",
    query_vector=query_vector,
    limit=5  # Return 5 closest points
)

In [9]:
from qdrant_client.models import Filter, FieldCondition, Range

hits = client.search(
    collection_name="clip_image_product",
    query_vector=query_vector,
    query_filter=Filter(
        must=[  # These conditions are required for search results
            FieldCondition(
                key='rand_number',  # Condition based on values of `rand_number` field.
                range=Range(
                    gte=3  # Select only those results where `rand_number` >= 3
                )
            )
        ]
    ),
    limit=5  # Return 5 closest points
)

In [13]:
client_grpc = QdrantClient(url="http://localhost:6334", prefer_grpc=True)

In [14]:
vectors = np.random.rand(100, 512)
client_grpc.upsert(
    collection_name="clip_image_product",
    points=[
        PointStruct(
            id=100+idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
        )
        for idx, vector in enumerate(vectors)
    ]
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
vectors = np.random.rand(100, 512)
client.upsert(
    collection_name="clip_image_product",
    points=[
        PointStruct(
            id=100+idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
        )
        for idx, vector in enumerate(vectors)
    ]
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
vectors = np.random.rand(100, 512)
client_grpc.upsert(
    collection_name="clip_image_product",
    points=[
        PointStruct(
            id=200+idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
        )
        for idx, vector in enumerate(vectors)
    ]
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
vectors = np.random.rand(100, 512)
client.upsert(
    collection_name="clip_image_product",
    points=[
        PointStruct(
            id=300+idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
        )
        for idx, vector in enumerate(vectors)
    ]
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
vectors = np.random.rand(100000000, 512)

In [ ]:
client_grpc.upsert(
    collection_name="clip_image_product",
    points=[
        PointStruct(
            id=400+idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
        )
        for idx, vector in enumerate(vectors)
    ]
)